In [76]:
import numpy as np

def calculate_hota(pred_boxes, gt_boxes, iou_threshold=0.5, c_threshold=0.5):
    """
    Calculate the Higher Order Tracking Accuracy (HOTA) metric.
    
    Args:
        pred_boxes (list): List of predicted bounding boxes.
        gt_boxes (list): List of ground truth bounding boxes.
        iou_threshold (float): Intersection over Union (IoU) threshold to consider a match.
        c_threshold (float): Confidence threshold for predicted boxes.
    
    Returns:
        float: HOTA score.
    """
    
    # Sort predicted boxes by confidence scores in descending order
    pred_boxes = sorted(pred_boxes, key=lambda x: x['confidence'], reverse=True)
    
    # Initialize variables for HOTA calculation
    HOTA = 0.0
    DetA = 0.0
    HOTAL = 0.0
    HOTAL_empty = 0.0
    mme = 0.0
    
    # Create boolean arrays to track matched predictions and ground truth boxes
    pred_matched = np.zeros(len(pred_boxes), dtype=bool)
    gt_matched = np.zeros(len(gt_boxes), dtype=bool)
    
    # Iterate over predicted boxes
    for p, pred_box in enumerate(pred_boxes):
        # Filter out low-confidence predictions
        if pred_box['confidence'] < c_threshold:
            break
        
        # Initialize variables for HOTA calculation per prediction
        best_iou = -np.inf
        best_match = -1
        
        # Iterate over ground truth boxes
        for g, gt_box in enumerate(gt_boxes):
            # Skip already matched ground truth boxes
            if gt_matched[g]:
                continue
            
            # Compute IoU between predicted and ground truth boxes
            iou = calculate_iou(pred_box['bbox'], gt_box['bbox'])
            
            # Check if IoU is higher than the threshold and better than previous matches
            if iou > iou_threshold and iou > best_iou:
                best_iou = iou
                best_match = g
        
        # Update match status for the best match (if any)
        if best_match != -1:
            pred_matched[p] = True
            gt_matched[best_match] = True
            
            # Calculate HOTA components for the matched prediction
            HOTA += best_iou
            DetA += 1
            HOTAL += 1
        else:
            # Calculate HOTA components for the unmatched prediction
            HOTAL_empty += 1
    
    # Calculate HOTA components for unmatched ground truth boxes
    HOTAL += len(gt_boxes) - np.count_nonzero(gt_matched)
    
    # Calculate the MME (Multiple Matches Error) component
    mme = np.count_nonzero(pred_matched) - DetA
    
    # Calculate the HOTA score
    HOTA_score = HOTA / (DetA + HOTAL + HOTAL_empty + mme)
    
    return [HOTA_score, DetA]


def calculate_iou(box1, box2):
    """
    Calculate the Intersection over Union (IoU) between two bounding boxes.
    
    Args:
        box1 (tuple): Bounding box coordinates of the format (x1, y1, x2, y2).
        box2 (tuple): Bounding box coordinates of the format (x1, y1, x2, y2).
    
    Returns:
        float: IoU score.
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = intersection / float(area_box1 + area_box2 - intersection)
    
    return iou


In [84]:
# Example usage
pred_boxes = [
    {'bbox': (70, 50, 150, 150), 'confidence': 0.8},
    {'bbox': (115, 120, 200, 200), 'confidence': 0.9},
    {'bbox': (100, 100, 210, 200), 'confidence': 0.9},
    # More predicted boxes...
]

gt_boxes = [
    {'bbox': (50, 50, 150, 150)},
    {'bbox': (100, 100, 200, 200)},
    {'bbox': (100, 100, 200, 200)},
    # More ground truth boxes...
]

hota_score = calculate_hota(pred_boxes, gt_boxes)
print("HOTA score:", hota_score)


HOTA score: [0.3981818181818182, 3.0]


In [39]:

import motmetrics as mm
import numpy as np

pred_boxes = [
    [1, 50, 50, 100, 100, 0.8],
    [2, 100, 100, 100, 100, 0.9],
    # More predicted boxes...
]

gt_boxes = [
    [1, 50, 50, 100, 100],
    [2, 100, 100, 100, 100],
    # More ground truth boxes...
]

mh = mm.metrics.create()

acc = mm.MOTAccumulator(auto_id=True)

# Create distance matrix


for pred_box, gt_box in zip(pred_boxes, gt_boxes,):
    pred_box = np.array(pred_box)[1:5]
    gt_box = np.array(gt_box)[1:]
    print(pred_box.shape, gt_box.shape)
    distances = mm.distances.iou_matrix(list(pred_box), list(gt_box), max_iou=0.5)
    print(type(distances))
    
    acc.update(pred_box, gt_box, distances)

metrics = mm.metrics.motchallenge_metrics
summary = mh.compute(acc, metrics)

hota_score = summary['HOTA'].global_track

print("HOTA score:", hota_score)




(4,) (4,)


IndexError: tuple index out of range

In [1]:
def motMetricsEnhancedCalculator(gtSource, tSource):
  # import required packages
  import motmetrics as mm
  import numpy as np
  
  # load ground truth
  gt = np.loadtxt(gtSource, delimiter=' ')

  # load tracking output
  t = np.loadtxt(tSource, delimiter=' ')

  # Create an accumulator that will be updated during each frame
  acc = mm.MOTAccumulator(auto_id=True)

  # Max frame number maybe different for gt and t files
  for frame in range(int(gt[:,0].max())):
    frame += 1 # detection and frame numbers begin at 1

    # select id, x, y, width, height for current frame
    # required format for distance calculation is X, Y, Width, Height \
    # We already have this format
    gt_dets = gt[gt[:,0]==frame,1:6] # select all detections in gt
    t_dets = t[t[:,0]==frame,1:6] # select all detections in t

    C = mm.distances.iou_matrix(gt_dets[:,1:], t_dets[:,1:], \
                                max_iou=0.5) # format: gt, t

    # Call update once for per frame.
    # format: gt object ids, t object ids, distance
    acc.update(gt_dets[:,0].astype('int').tolist(), \
              t_dets[:,0].astype('int').tolist(), C)

  mh = mm.metrics.create()

  summary = mh.compute(acc, metrics=['num_frames', 'idf1', 'idp', 'idr', \
                                     'recall', 'precision', 'num_objects', \
                                     'mostly_tracked', 'partially_tracked', \
                                     'mostly_lost', 'num_false_positives', \
                                     'num_misses', 'num_switches', \
                                     'num_fragmentations', 'mota', 'motp' \
                                    ], \
                      name='acc')

  strsummary = mm.io.render_summary(
      summary,
      #formatters={'mota' : '{:.2%}'.format},
      namemap={'idf1': 'IDF1', 'idp': 'IDP', 'idr': 'IDR', 'recall': 'Rcll', \
               'precision': 'Prcn', 'num_objects': 'GT', \
               'mostly_tracked' : 'MT', 'partially_tracked': 'PT', \
               'mostly_lost' : 'ML', 'num_false_positives': 'FP', \
               'num_misses': 'FN', 'num_switches' : 'IDsw', \
               'num_fragmentations' : 'FM', 'mota': 'MOTA', 'motp' : 'MOTP',  \
              }
  )
  print(strsummary)

In [3]:
motMetricsEnhancedCalculator('data/anno_test_track/2020_12_16_Montana_at_Washington_gt_xywh.txt', 'hota/2020_12_16_Montana_at_Washington.txt')

     num_frames      IDF1       IDP       IDR      Rcll      Prcn    GT  MT  PT  ML  FP  FN  IDsw  FM      MOTA      MOTP
acc         200  0.786342  0.797668  0.775333  0.945333  0.972565  1500  10   0   0  40  82    23  12  0.903333  0.089832


In [74]:
motMetricsEnhancedCalculator('data/anno_test_track/2021_01_16_Citadel_at_VirginiaMilitary_gt.txt', 'hota/2021_01_16_Citadel_at_VirginiaMilitary.txt')



     num_frames      IDF1       IDP       IDR      Rcll      Prcn    GT  MT  PT  ML    FP    FN  IDsw  FM      MOTA      MOTP
acc         193  0.025357  0.023278  0.027842  0.059165  0.049467  1724   0   0  10  1960  1622    32  39 -1.096288  0.396978


# Пробуем посчитать метрику HOTA с использованием библиотеки Trackeval.(!разобраться с форматом подаваемых данных!)

In [69]:
import trackeval
import numpy as np

gt_data_root = "data/anno_test_track/2020_12_16_Montana_at_Washington_gt.txt"
pr_data_root = "hota/2020_12_16_Montana_at_Washington.txt"

evaluator = trackeval.Evaluator()

#predicted_data = trackeval.datasets.MotChallenge2DBox(pr_data_root)
#groundtruth_data = trackeval.datasets.MotChallenge2DBox(gt_data_root)

hota = trackeval.metrics.HOTA()

data = {'num_tracker_dets':[
    [1, 50, 50, 100, 100, 0.8],
    [2, 100, 100, 100, 100, 0.9],
  ],
  'num_gt_dets': [
    [1, 50, 50, 100, 100],
    [2, 100, 100, 100, 100],
    # More ground truth boxes...
  ],
  'num_gt_ids': 2,
  'num_tracker_ids': 2,
  'gt_ids':[1,2],
  'tracker_ids': [1,2]}

hota.eval_sequence(data)


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /home/skorp/Projects/.env/lib/python3.10/site-packages/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : True                          
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          


KeyError: 'similarity_scores'